In [141]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import cv2

이 데이터세트는 CNN을 이용해서 해결해보았다. COVID-19가 걸린 사람과 걸리지 않은 사람에 대한 폐의 x-ray사진을 가지고 test_data, train_data, validation_data로 나누어서 tf.keras.layers.Sequential()을 이용해서 신경망 모델을 생성하여서 학습을 했다.

이 데이터셋의 경우 test와 train으로 나누어져있기는 했지만 그냥 전체 이미지 데이터에서 train, test, val을 train_test_split으로 나누었다.

처음에 이 데이터셋은 완전히 .jpeg의 형태로 저장되어 있었기 때문에 이를 import os를 해서 저장된 directory를 os.path.join이라는 메서드를 이용해서 list에 저장했다.

이 list는 이후 keras.model에 적용하기 위해서 type와 shape를 바꿔주는 것이 중요하다.

현재 작업 폴더 얻기: 	os.getcwd()

디렉토리 변경:	os.chdir("C:\Tmp")

특정 경로에 대해 절대 경로 얻기:	   os.path.abspath(".\\Scripts")

경로 중 디렉토리명만 얻기:	  os.path.dirname("C:/Python35/Scripts/pip.exe")

경로 중 파일명만 얻기:	  if os.path.isfile("C:/Python35/Scripts/pip.exe"):
                               print(os.path.basename("C:/Python35/Scripts/pip.exe"))
                               
경로 중 디렉토리명과 파일명을 나누어 얻기:	  dir, file = os.path.split("C:/Python35/Scripts/pip.exe")

파일 각 경로를 나눠 리스트로 리턴하기
os.path.sep은 OS별 경로 분리자	"C:\Python35\Scripts\pip.exe".split(os.path.sep)

경로를 병합하여 새 경로 생성:	os.path.join('C:\Tmp', 'a', 'b')

디렉토리 안의 파일/서브디렉토리 리스트:	os.listdir("C:\Python35")

파일 혹은 디렉토리 경로가 존재하는지 체크하기:	os.path.exists("C:\Python35")

디렉토리 경로가 존재하는지 체크하기:   os.path.isdir("C:\Python35")

파일 경로가 존재하는지 체크하기:	os.path.isfile("C:\Python35\python.exe")

파일의 크기: 	os.path.getsize("C:\Python35\python.exe")


In [142]:
categories = ['normal', 'pneumonia']
train_dir = "C:/xray_dataset_covid19/train"
test_dir = "C:/xray_dataset_covid19/test"
new_size = 128
x_test_data, x_train_data = [],[]
y_test_data, y_train_data = [],[]

In [143]:
for category in categories:
    label = categories.index(category)
    path_train = os.path.join(train_dir, category)
    path_test = os.path.join(test_dir, category)
    for img in os.listdir(path_train):
        try:
            img_train = cv2.imread(os.path.join(path_train,img), cv2.IMREAD_COLOR)
            img_train = cv2.resize(img_train, (new_size, new_size))
            x_train_data.append(img_train)
            y_train_data.append(label)
        except Exception as e:
            pass
    for img in os.listdir(path_test):
        try:
            img_test = cv2.imread(os.path.join(path_test,img), cv2.IMREAD_COLOR)
            img_test = cv2.resize(img_test, (new_size, new_size))
            x_test_data.append(img_test)
            y_test_data.append(label)
        except Exception as e:
            pass

In [159]:
x_data = x_train_data+x_test_data
y_data = y_train_data+y_test_data

x_test_data = np.array(x_test_data, dtype="float32").reshape(-1, new_size, new_size, 3)

x_train_data = np.array(x_train_data, dtypes = 'float32).reshape(-1, new_size, new_size, 3)

y_test_data = np.asarray(y_test_data)

y_train_data = np.asasrray(y_train_data)

이런 식으로 바꾸어 주어야 한다. 이를 하나의 pipeline으로 만들 수도 있지 않을까?


image가 저장된 directory뿐만 아니라 실제로 데이터를 불러 올 수 있어야 한다.

따라서 os와 os.path를 이용해서 해당 이미지의 이름을 모두 image_data라는 리스트에 저장해 보았다.

처음에는 x_data의 자료형을 알아보기 위해서 .shape()를 사용해 보았지만 list는 shape를 적용할 수 없고 np.array로 모양을 바꾼 이후에서는 shape가 측정이 가능하다.

In [160]:
x_data = np.array(x_data, dtype = 'float32').reshape(-1, new_size, new_size, 3)

y_data = np.asarray(y_data)

x_data.shape

(188, 128, 128, 3)

In [161]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

index = np.arange(x_data.shape[0])
np.random.shuffle(index)

x_data = x_data[index]
y_data = y_data[index]

y_cat = to_categorical(y_data)

(x_train, x_test, y_train, y_test) = train_test_split(x_data, y_cat, test_size=0.2, random_state=None, shuffle= False, stratify=None) 
(x_train, x_val , y_train, y_val ) = train_test_split(x_train, y_train, test_size=0.25, random_state=None, shuffle = False, stratify=None)


In [162]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=10, width_shift_range=0.2,
    height_shift_range=0.2, shear_range=0.5, zoom_range=0.3,horizontal_flip=True,
    vertical_flip=False, rescale=1./255, validation_split=0.2, dtype=None
)

test_datagen = ImageDataGenerator(
    rescale=1./255, samplewise_center = True
)

validation_datagen = ImageDataGenerator(
    rescale = 1./255, samplewise_center = True
)


batch_size = 16
epochs = 100

train_iterator = train_datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=False)
validation_iterator = validation_datagen.flow(x_val, y_val, batch_size=batch_size, shuffle=False)
test_iterator = test_datagen.flow(x_test, y_test, batch_size=len(x_test), shuffle=False)

In [163]:
model = tf.keras.models.Sequential()
#First Convolution Layer
model.add(tf.keras.layers.Conv2D(filters = 32, input_shape = (128,128,3), activation = 'relu', kernel_size = 3))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2), strides = (2,2)))
#Second Convolution Layer
model.add(tf.keras.layers.Conv2D(filters = 64, activation = 'relu', kernel_size = 3))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2), strides = (2,2)))
#Third Convolution Layer + BatchNorm
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters=128, activation = 'relu', kernel_size = 3))
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2), strides = (2,2)))
#Dropout + Flatten
model.add(tf.keras.layers.Dropout(rate = 0.5))
model.add(tf.keras.layers.Flatten())
#Dense Layer(using sigmoid)
model.add(tf.keras.layers.Dense(units = 256, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 64, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 2, activation = 'softmax'))


In [164]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 30, 30, 64)        256       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 28, 28, 128)       73856     
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 14, 14, 128)     

In [165]:
model.compile(
    loss='binary_crossentropy', 
    optimizer= tf.keras.optimizers.Adam(lr = 0.0001),
    metrics = ['accuracy']
)

In [166]:
early_stopping_callbacks = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 100)
history = model.fit_generator(
    train_iterator,
    validation_data = validation_iterator, 
    steps_per_epoch = x_train.shape[0]//batch_size,
    epochs = epochs,
    callbacks = [early_stopping_callbacks]
)

Epoch 1/100
7/7 [==============================] - 4s 522ms/step - loss: 0.6679 - accuracy: 0.6429 - val_loss: 0.6854 - val_accuracy: 0.5526
Epoch 2/100
7/7 [==============================] - 3s 490ms/step - loss: 0.3940 - accuracy: 0.8036 - val_loss: 0.6663 - val_accuracy: 0.6579
Epoch 3/100
7/7 [==============================] - 3s 470ms/step - loss: 0.4142 - accuracy: 0.8036 - val_loss: 0.6531 - val_accuracy: 0.7105
Epoch 4/100
7/7 [==============================] - 4s 504ms/step - loss: 0.2512 - accuracy: 0.8839 - val_loss: 0.6561 - val_accuracy: 0.6316
Epoch 5/100
7/7 [==============================] - 3s 495ms/step - loss: 0.3400 - accuracy: 0.8750 - val_loss: 0.6579 - val_accuracy: 0.6579
Epoch 6/100
7/7 [==============================] - 4s 510ms/step - loss: 0.3084 - accuracy: 0.8661 - val_loss: 0.6628 - val_accuracy: 0.6579
Epoch 7/100
7/7 [==============================] - 3s 493ms/step - loss: 0.3623 - accuracy: 0.8571 - val_loss: 0.6496 - val_accuracy: 0.6053
Epoch 8/100
7

 from tensorflow.keras.callbacks import ReduceLROnPlateau
 
 learning_rate_reduction = ReduceLROnPlateau(
     monitor='accuracy', 
     patience=3,
     verbose=1,
     factor=0.5, 
     min_lr=0.00001)
                                                
 이런 식으로 중간에 'accuracy'값을 'patience' = 3로 관찰하면서 optimizer의 learning_rate를 줄여 나가는 것이 가능하다.
                                                
                                                

In [167]:
predicted_label = model.predict(x_test)
model.evaluate(x_test,y_test)

38/1 [==================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[2.3857623526924536, 0.84210527]

정확도는 84.21%정도 나왔다.